<a href="https://colab.research.google.com/github/lauratcc18/IAMINTIC/blob/main/nlp_005_clase_ir_propuestas_bogota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP - IR
### Luis Gabriel Moreno Sandoval
### morenoluis@javeriana.edu.co

In [ ]:
!pip install Spacy
!python -m spacy download es_core_news_sm

2023-09-28 01:52:15.190514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 01:52:17.358219: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-28 01:52:19.256178: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-28 01:52:19.256649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import SnowballStemmer
import nltk
import es_core_news_sm
nltk.download('stopwords')
nltk.download('punkt')
stop_words_spanish = nltk.corpus.stopwords.words('spanish') # english
stemmer = SnowballStemmer('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def pre_process(text):
  # print('before pre_process', text)
  text = text.lower()
  # print('after pre_process', text)
  return text

In [ ]:
def tokenize_stemmer(text):
  tokens_doc = nltk.word_tokenize(text)
  tokens_doc_wo_stop = [w for w in tokens_doc if w.isalpha()]
  tokens_docs_stem = [stemmer.stem(w) for w in tokens_doc_wo_stop]
  return tokens_docs_stem

In [ ]:
def tokenize_lemma(text):
  nlp = es_core_news_sm.load()
  doc = nlp(text)
  words = [word.lemma_.lower() for word in doc if word.text.isalpha()]
  return words

In [ ]:
from nltk.corpus import PlaintextCorpusReader
corpus_root = '/content/planes'
wordlists = PlaintextCorpusReader(corpus_root, '.*')
print(wordlists.fileids())
print(wordlists.words()[500:550])

['2023_Bolivar.txt', '2023_Galan.txt', '2023_Lara.txt', '2023_Molano.txt', '2023_Robledo.txt', '2023_Vargas.txt']
['Comisión', 'Tercera', 'Constitucional', 'del', 'Senado', '.', 'Es', 'autor', 'de', 'más', 'de', '8', 'libros', 'y', 'más', 'de', '3', '.', '800', 'guiones', '.', 'Fue', 'asistente', 'del', 'exministro', 'de', 'justicia', 'Enrique', 'Parejo', 'González', 'en', 'el', 'Concejo', 'de', 'Bogotá', ',', 'presidió', 'la', 'Fundación', 'Manos', 'Limpias', ',', 'ejerció', 'como', 'activista', 'ciudadano', 'contra', 'la', 'corrupción', 'y']


In [ ]:
stop_words_spanish.append('aaa')
stop_words_spanish[:10]

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se']

In [ ]:
docs_raw = {}
for item in wordlists.fileids():
  docs_raw[item] = wordlists.raw(item)
docs_raw.keys()

dict_keys(['2023_Bolivar.txt', '2023_Galan.txt', '2023_Lara.txt', '2023_Molano.txt', '2023_Robledo.txt', '2023_Vargas.txt'])

In [ ]:
tfidf = TfidfVectorizer(preprocessor=pre_process, tokenizer=tokenize_stemmer, stop_words=stop_words_spanish)
tfs = tfidf.fit_transform(docs_raw.values())
print(tfidf.get_feature_names_out())
print(len(tfidf.get_feature_names_out()))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['algun', 'com', 'contr', 'cuand', 'desd', 'dond', 'durant', 'eram', 'estab', 'estais', 'estam', 'estan', 'estand', 'estaran', 'estaras', 'esteis', 'estem', 'esten', 'estes', 'estuv', 'fuer', 'fues', 'fuim', 'fuist', 'hab', 'habr', 'habran', 'habras', 'hast', 'hem', 'hub', 'mas', 'mia', 'mias', 'mio', 'mios', 'much', 'nad', 'nosotr', 'nuestr', 'par', 'per', 'poc', 'porqu', 'qui', 'seais', 'seam', 'sent', 'ser', 'seran', 'seras', 'si', 'sient', 'sint', 'sobr', 'som', 'suy', 'tambien', 'tant', 'ten', 'tendr', 'tendran', 'tendras', 'teng', 'tien', 'tod', 'tuv', 'tuy', 'vosotr', 'vuestr'] not in stop_words.
  wa

['aban' 'abander' 'abandon' ... 'ﬁrmad' 'ﬁscal' 'ﬂexibl']
4093


In [ ]:
docs_num, feature_num = tfs.shape
feature_names = tfidf.get_feature_names_out()
print("n_docs: %d, n_features: %d" % tfs.shape)

n_docs: 6, n_features: 4093


In [ ]:
print("###### Calculo de Feature Names ######")
for x in range(0, feature_num):
    print(" # ", x ," - ",feature_names[x], " \t - ", [tfs[n,x] for n in range(0, docs_num)])
    if x == 100:
      break

###### Calculo de Feature Names ######
 #  0  -  aban  	 -  [0.0, 0.0, 0.0, 0.0, 0.007265932377241132, 0.0]
 #  1  -  abander  	 -  [0.0, 0.0, 0.0, 0.004873854126320782, 0.0, 0.0]
 #  2  -  abandon  	 -  [0.0, 0.0, 0.00868066485965739, 0.002891458540584962, 0.004310580842793141, 0.0025401938164019227]
 #  3  -  abarc  	 -  [0.0030502890384767442, 0.0, 0.0, 0.0, 0.0, 0.007022210373589733]
 #  4  -  abast  	 -  [0.0, 0.0, 0.0, 0.0, 0.0, 0.004281760896775777]
 #  5  -  abastec  	 -  [0.018026845215011928, 0.0, 0.022792564324442297, 0.0, 0.0, 0.011857265035275753]
 #  6  -  abiert  	 -  [0.009907300075053582, 0.0021077164312540194, 0.01461421748917692, 0.00649050192294672, 0.009676027809655562, 0.00570201253747826]
 #  7  -  abog  	 -  [0.0, 0.0, 0.008998937566329926, 0.003996630105032836, 0.0, 0.0]
 #  8  -  abord  	 -  [0.011034026241820311, 0.0, 0.015191163504400433, 0.011565834162339849, 0.0, 0.01778135671481346]
 #  9  -  abordaj  	 -  [0.0, 0.0028169044930413402, 0.0, 0.0028914585405

In [ ]:
response = tfidf.transform([wordlists.raw('2023_Galan.txt')])

print('response:', response)
cosine_similarity_response =  cosine_similarity(response, tfs)
print("n_question: %d, n_features: %d" % response.shape)
print("cosine_similarity ", cosine_similarity_response)


response:   (0, 4080)	0.01459573411208944
  (0, 4079)	0.031615746468810255
  (0, 4074)	0.0047481855244182625
  (0, 4073)	0.0047481855244182625
  (0, 4066)	0.0047481855244182625
  (0, 4064)	0.01459573411208944
  (0, 4062)	0.0038935800537584983
  (0, 4059)	0.0038935800537584983
  (0, 4056)	0.008450713479124012
  (0, 4051)	0.009496371048836525
  (0, 4050)	0.009730489408059624
  (0, 4048)	0.016861731450032134
  (0, 4046)	0.0038935800537584983
  (0, 4044)	0.002107716431254017
  (0, 4043)	0.002816904493041337
  (0, 4042)	0.0047481855244182625
  (0, 4041)	0.002816904493041337
  (0, 4039)	0.00729786705604472
  (0, 4037)	0.0047481855244182625
  (0, 4034)	0.0038935800537584983
  (0, 4033)	0.00729786705604472
  (0, 4030)	0.0032872278226746708
  (0, 4029)	0.018969447881286152
  (0, 4026)	0.008430865725016067
  (0, 4024)	0.0047481855244182625
  :	:
  (0, 62)	0.0252925971750482
  (0, 59)	0.002432622352014906
  (0, 58)	0.010538582156270085
  (0, 57)	0.005633808986082674
  (0, 56)	0.004865244704029812

In [ ]:
wordlists.fileids()

['2023_Bolivar.txt',
 '2023_Galan.txt',
 '2023_Lara.txt',
 '2023_Molano.txt',
 '2023_Robledo.txt',
 '2023_Vargas.txt']

In [ ]:
tfidf = TfidfVectorizer(preprocessor=pre_process, tokenizer=tokenize_lemma, stop_words=stop_words_spanish)
tfs = tfidf.fit_transform(docs_raw.values())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aao', 'alguno', 'erai', 'estarer', 'estaríar', 'ester', 'estuvir', 'estuviser', 'estuvistir', 'estuviés', 'estuviéser', 'estár', 'estéi', 'fuerai', 'fuistei', 'fuistir', 'fuérar', 'fués', 'haber', 'habríai', 'habríar', 'habíai', 'hayar', 'hubierai', 'hubir', 'hubiérar', 'hubiés', 'hubiésar', 'ir', 'nosotra', 'sentir', 'ser', 'serer', 'seríai', 'seái', 'tendréi', 'tendríai', 'tener', 'teníai', 'tuvierai', 'tuviérir', 'tuviés', 'vosotra', 'vosotro', 'vuestrir'] not in stop_words.
  warnings.warn(


In [ ]:
docs_num, feature_num = tfs.shape
feature_names = tfidf.get_feature_names_out()
print("n_docs: %d, n_features: %d" % tfs.shape)

n_docs: 6, n_features: 5746


In [ ]:
print("###### Calculo de Feature Names ######")
for x in range(0, feature_num):
    print(" # ", x ," - ",feature_names[x], " \t - ", [tfs[n,x] for n in range(0, docs_num)])
    if x == 100:
      break

###### Calculo de Feature Names ######
 #  0  -  aao  	 -  [0.0, 0.006166716189602489, 0.0, 0.0, 0.0, 0.0]
 #  1  -  abanderar  	 -  [0.0, 0.0, 0.0, 0.00680460295706886, 0.0, 0.0]
 #  2  -  abandonado  	 -  [0.0, 0.0, 0.00436602072924996, 0.0, 0.0, 0.0]
 #  3  -  abandonar  	 -  [0.0, 0.0, 0.007160398909554618, 0.005579871765991163, 0.0, 0.0]
 #  4  -  abandono  	 -  [0.0, 0.0, 0.0030226503875547726, 0.0, 0.005599563977536613, 0.003600400451528722]
 #  5  -  abanico  	 -  [0.0, 0.0, 0.0, 0.0, 0.008088203816539864, 0.0]
 #  6  -  abarcar  	 -  [0.0038872263944597116, 0.0, 0.0, 0.0, 0.0, 0.008529039141685736]
 #  7  -  abastecer  	 -  [0.00474043729459043, 0.0, 0.0, 0.0, 0.0, 0.0]
 #  8  -  abastecimiento  	 -  [0.019691181761476967, 0.0, 0.02720385348799295, 0.0, 0.0, 0.014401601806114888]
 #  9  -  abasto  	 -  [0.0, 0.0, 0.0, 0.0, 0.0, 0.005200542897616361]
 #  10  -  abierto  	 -  [0.012625661982778223, 0.002737401260515779, 0.017442663570368624, 0.009061676331106005, 0.0107710450628

In [ ]:
response = tfidf.transform([wordlists.raw('2023_Galan.txt')])

print('response:', response)
cosine_similarity_response =  cosine_similarity(response, tfs)
print("n_question: %d, n_features: %d" % response.shape)
print("cosine_similarity ", cosine_similarity_response)

response:   (0, 5730)	0.01368700630257887
  (0, 5727)	0.016424407563094642
  (0, 5723)	0.006166716189602477
  (0, 5720)	0.010949605042063095
  (0, 5716)	0.010975382355513732
  (0, 5713)	0.010949605042063095
  (0, 5712)	0.012333432379204954
  (0, 5711)	0.030111413865673516
  (0, 5710)	0.014633843140684976
  (0, 5707)	0.010975382355513732
  (0, 5705)	0.015170386474716071
  (0, 5703)	0.018956241154705646
  (0, 5702)	0.04106101890773661
  (0, 5696)	0.006166716189602477
  (0, 5689)	0.006166716189602477
  (0, 5686)	0.02737401260515774
  (0, 5684)	0.016424407563094642
  (0, 5682)	0.018500148568807433
  (0, 5681)	0.0031593735257842742
  (0, 5680)	0.0063187470515685485
  (0, 5676)	0.005056795491572024
  (0, 5674)	0.004269294223814728
  (0, 5672)	0.004269294223814728
  (0, 5671)	0.010113590983144048
  (0, 5664)	0.012637494103137097
  :	:
  (0, 80)	0.03558621638670506
  (0, 76)	0.010113590983144048
  (0, 75)	0.010113590983144048
  (0, 72)	0.007316921570342488
  (0, 71)	0.006166716189602477
  (0, 

In [ ]:
response

<1x136 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [ ]:
!pip install joblib


In [ ]:
import pandas as pd
dict_data = []
count = 0
for item in docs_raw:
  dict_data.append({'index': count, 'text': item})
  count += 1
df = pd.DataFrame.from_records(dict_data) # columns=['index', 'text']
df

,index,text
0,0,El precio del dólar en el mercado interbancari...
1,1,"Helicóptero del Ejército, que transportaba una..."
2,2,"Los préstamos de dinero a partir del crédito, ..."
3,3,"Samuel Santander Lopesierra, más conocido como..."


In [ ]:
from joblib import dump
from datetime import datetime
data_filename_tf_idf = f"{datetime.now().strftime('%Y-%m-%d')}_file_tfidf.vec"
data_filename_tfs = f"{datetime.now().strftime('%Y-%m-%d')}_file_tfs.pkl"
data_filename_df = f"{datetime.now().strftime('%Y-%m-%d')}_file_df_docs.pkl"
dump(tfidf, data_filename_tf_idf)
dump(tfs, data_filename_tfs)
dump(df, data_filename_df)

['2023-09-28_file_df_docs.pkl']

In [ ]:
from joblib import load
tf_idf_2_re = load('2023-09-28_file_tfidf.vec', mmap_mode='r')

In [ ]:
response = tf_idf_2_re.transform(["Hay un articulo que hable del precio del dolar."])
print('response:', response)
cosine_similarity_response =  cosine_similarity(response, tfs)
print("n_question: %d, n_features: %d" % response.shape)
print("cosine_similarity ", cosine_similarity_response)

response:   (0, 100)	0.7071067811865476
  (0, 37)	0.7071067811865476
n_question: 1, n_features: 136
cosine_similarity  [[0.44934408 0.         0.         0.        ]]


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['algun', 'com', 'contr', 'cuand', 'desd', 'dond', 'durant', 'eram', 'estab', 'estais', 'estam', 'estan', 'estand', 'estaran', 'estaras', 'esteis', 'estem', 'esten', 'estes', 'estuv', 'fuer', 'fues', 'fuim', 'fuist', 'hab', 'habr', 'habran', 'habras', 'hast', 'hem', 'hub', 'mas', 'mia', 'mias', 'mio', 'mios', 'much', 'nad', 'nosotr', 'nuestr', 'par', 'per', 'poc', 'porqu', 'qui', 'seais', 'seam', 'sent', 'ser', 'seran', 'seras', 'si', 'sient', 'sint', 'sobr', 'som', 'suy', 'tambien', 'tant', 'ten', 'tendr', 'tendran', 'tendras', 'teng', 'tien', 'tod', 'tuv', 'tuy', 'vosotr', 'vuestr'] not in stop_words.
  warnings.warn(


In [ ]:
import unicodedata
def proper_encoding(text):
    # print('text: ', text)
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return text

In [ ]:
text = "Holá niño otro lingüística."
print(proper_encoding(text))

Hola nino otro linguistica.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')